I have placed some of the less interesting code in the `convert_book_to_text.py` file to try to keep the notebook flowing

In [3]:
##for the reload of all the functions in the file convert_pdf_to_text.py
from importlib import reload
import convert_book_to_text

reload(convert_book_to_text)
from convert_book_to_text import find_chapter_pages, \
                                 find_page_starting_with,\
                                 extract_text_from_pages_inclusive, \
                                 build_chapter_dictionary, \
                                 remove_chapter_header, \
                                 chunk_text, \
                                 find_bad_csv_files


For copyright reasons I have not included the ebook in the repo. You should buy your own copy if you want to run this code

In [4]:
from pathlib import Path
import PyPDF2

input_file = Path('./inputs/Truth to Power_ My Three Years Inside Eskom.pdf')
input_file = Path(input_file)
# Instantiate PdfReader object
pdf = PyPDF2.PdfReader(input_file)


In [5]:
# This is very specific to the format of the book I am precessing. Not making this general is a design decision. This repo needs back and forward with a human
# to work so spending too much time generalising functions seems like a waste of time.

# Note: This is quite slow. It takes about 30 seconds to run on my machine.
pages_with_numbers = find_chapter_pages(pdf)
pages_with_numbers.append(find_page_starting_with(pdf, "Index"))
# Create a dictionary where each chapter is saved with its start and end page
chapter_start_and_end = build_chapter_dictionary(pages_with_numbers)

The following cell is for illustration. It allows the user to see what is happening in the main loop below, but for a single chapter

The main loop. Chapters start at 1 and end at 31. chapter_range_end should be set to the number after the last chapter you want to process so, if you are processing the last chapter, chapter 31, only you would set `chapter_range_start = 31` and `chapter_range_end = 32`.

The output of this is a list of CSV files in the folder `./output/`. A separate file is created for each chunk. 

The function sets a chunk size of 800 words. There is no science in determining how long this should be but we need to ensure there are sufficient tokens for the input, the system command and the output. I started with a chunk size of 1000 words but I found there were times where GPT3.5 would truncate the output. 800 seemed reasonable. 

In [88]:
chapter_range_start = 28
chapter_range_end = 32
save_people_named_in_chapters(chapter_range_start=chapter_range_start, chapter_range_end=chapter_range_end, chapter_start_and_end = chapter_start_and_end, pdf = pdf)

processing chapter:  28 from page 204 to page 208
processing chunk 0 out of 3
processing chunk 1 out of 3
processing chunk 2 out of 3
processing chapter:  29 from page 209 to page 212
processing chunk 0 out of 3
processing chunk 1 out of 3
processing chunk 2 out of 3
processing chapter:  30 from page 213 to page 218
processing chunk 0 out of 4
processing chunk 1 out of 4
processing chunk 2 out of 4
processing chunk 3 out of 4
processing chapter:  31 from page 219 to page 226
processing chunk 0 out of 6
processing chunk 1 out of 6
processing chunk 2 out of 6
processing chunk 3 out of 6
processing chunk 4 out of 6
processing chunk 5 out of 6


It is hard to ensure the LLM creates output in a consistent format (it doesn't matter how nicely you ask). There are two approaches to work with this, one would be to run a second ChatGPT call to check if the output was formatted correctly and if not to fix it, the other is just to do it manually. Given that the number of times GPT provided output in a different format, I opted just to fix the issues manually.

In [5]:
find_bad_csv_files('./outputs')

[('all.csv', 'Columns mismatch'), ('all_v1.csv', 'Columns mismatch')]

Once all the output files are formatted correctly, they are concatenated into a single list

In [92]:
def combine_csvs(directory):
    directory_path = Path(directory)
    csv_files = directory_path.glob('*.csv')

    df_list = []

    for filepath in csv_files:
        df = pd.read_csv(filepath, encoding='utf-8', delimiter='|')
        df.columns = df.columns.str.strip()  # remove leading and trailing whitespaces from column names
        df = df[df["Name"].str.strip() != "-----"] # remove rows which had markdown formatting        
        df = df[df["Name"].str.strip() != "----"] # remove rows which had markdown formatting        
        df = df[df["Name"].str.strip() != "---"] # remove rows which had markdown formatting
        df = df[df["Name"].str.strip() != "-"] # remove rows which had markdown formatting
        df["Chapter"] = filepath.stem
        df_list.append(df)

    combined_df = pd.concat(df_list, ignore_index=True)
    
    return combined_df

In [6]:
# output the number of rows in df
df = combine_csvs('./outputs')
print(df.shape)
df.to_csv("./outputs/all.csv", encoding='utf-8', index=False)

NameError: name 'combine_csvs' is not defined

The real manual work begins here. The consolidated list needs to be cleaned. Even though the system prompt asked only for information on people, the AI added organisations. Sometimes it would only list a person by their first or last name, sometimes both. Sometimes the list would include a title, other times it did not. I opted for a manual exercise to remove items I was not interested in and to ensure the names that were left were realitvely standard. 

In [2]:
import pandas as pd
manual_list = pd.read_csv("./outputs/all_v1.csv", encoding='utf-8')

In [4]:
# count the number of unique names in the column manual_list["Name"]
manual_list["Name"].nunique()

280

In [3]:
import openai
# get a list of all elements in manual_list that have "name" equal to "Cyril Ramaphosa"
name = "ANC"
filename = "./aggregate/" + name + ".txt"
subset = manual_list[manual_list["Name"] == name]
if len(subset) > 1:
    print("Found " + str(len(subset)) + " entries for " + name)
    pipe_delimited_string = subset.to_csv(encoding='utf-8', index=False, sep='|')

    system_instruction = "The user will provide a table with the sentiment analysis for one person or organization from various places in a book. Scores in the Sentiment column are on a scale from 0 (extremely negative) to 10 (extremely positive). Please provide one overall sentiment analysis and one consolidated, summary Reason for the person or organization by aggregating all these into a single item. The one aggregate Sentiment score and reason should weight Sentiment Scores more highly if the Reason they have is compelling. The one aggregate Sentiment score should discount Neutral Ratings or ratings that correspond to Reasons that are not compelling. The aggregate Reason should be a short summary that supports the score. Do not provide any text other than the Aggregate Score and Aggregate Reason."
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_instruction},
            {"role": "user", "content": pipe_delimited_string},
        ]
    )
    output = response['choices'][0]['message']['content']
    with open(filename, "w", encoding='utf-8') as f:
        f.write(output)

elif len(subset) == 1:
    print("Found one entry for " + name)
    # create the output string using subset["Sentiment"] and subset["Reason"]
    output = "Sentiment: " + str(subset.loc["Sentiment"][0]) + "\nReason: " + subset.loc["Reason"][0]
    with open(filename, "w", encoding='utf-8') as f:
        f.write(output)
else:
    print("No entry found for " + name)
    output = ""

print(output)



Found 7 entries for ANC
Aggregate Sentiment Score: 1

Aggregate Reason: The overall sentiment towards the ANC is negative, with the text criticising the party for being stuck in the past, lacking business acumen leading to counterproductive legislation, corruption, and being politically sensitive towards dealing with issues. The sentiment is also negatively impacted due to the party's strict procurement and labour laws pushing investors away, and the general negative opinion about politicians' unwillingness to hear the truth.
